# n8n Workflow Generator - Fine-tuning Llama 3 8B

This notebook demonstrates how to fine-tune Llama 3 8B to generate n8n workflow configurations from natural language descriptions.

**Related Resources:**
- 📝 Blog Post Part 1: [what-are-people-actually-building-in-n8n-i-scraped-over-6-000-workflows-to-find-out](https://medium.com/@mustaphaliaichi/what-are-people-actually-building-in-n8n-i-scraped-over-6-000-workflows-to-find-out-59eb8e34c317)
- 📝 Blog Post Part 2: [Link]
- 🤗 Model: [MustaphaL/n8n-workflow-generator](https://huggingface.co/MustaphaL/n8n-workflow-generator)
- 🤗 Dataset: [MustaphaL/n8n-workflow-training-data](https://huggingface.co/datasets/MustaphaL/n8n-workflow-training-data)
- 🌐 Community: [n8nlearninghub.com](https://n8nlearninghub.com)

**Hardware Requirements:**
- GPU: A100 (Google Colab Pro - $9.99/month)
- RAM: 12GB+
- Training time: ~55 minutes

In [1]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 144.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 20.7 MB/s eta 0:00:00


In [2]:
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0


In [ ]:
from google.colab import files
print("Upload your training_data_alpaca.json file:")
uploaded = files.upload()

Upload your training_data_alpaca.json file:


In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

# Load model (same as before)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Add LoRA (same as before)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# Load data
dataset = load_dataset('json', data_files='training_data_alpaca.json')

# Format data (same as before)
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

# ⚡ NEW BETTER TRAINING ARGS - ANTI-OVERFITTING
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,              # ← MORE warmup
        num_train_epochs = 1.5,          # ← LESS epochs! (1.5 instead of 3)
        learning_rate = 5e-5,            # ← LOWER learning rate
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.1,              # ← MORE regularization
        lr_scheduler_type = "cosine",    # ← Better scheduler
        seed = 3407,
        output_dir = "outputs",
        max_grad_norm = 0.5,             # ← Gradient clipping
        save_strategy = "steps",         # ← Save checkpoints
        save_steps = 500,                # ← Every 500 steps
        eval_strategy = "no",
    ),
)

# Train
print("🚀 Starting IMPROVED training...")
print("Target: Final loss between 0.7-1.5")
trainer_stats = trainer.train()

# Save
print("💾 Saving model...")
model.save_pretrained("n8n_generator_v2")
tokenizer.save_pretrained("n8n_generator_v2")

print("✅ Training complete!")
print(f"Time: {trainer_stats.metrics['train_runtime']:.0f} seconds")

In [ ]:
from unsloth import FastLanguageModel

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "n8n_generator_v2",  # Your saved model
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

instruction = """Create an n8n workflow for: Build a Telegram chatbot that uses OpenAI to respond to messages"""

inputs = tokenizer(
    [alpaca_prompt.format(instruction, "")],
    return_tensors = "pt"
).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.7,
    top_p = 0.9,
    use_cache = True
)

# BETTER DECODING - Removes special tokens
result = tokenizer.decode(outputs[0], skip_special_tokens=True)  # ← skip_special_tokens=True

# Extract just the Response part
response = result.split("### Response:")[-1].strip()

print(response)

In [ ]:
from unsloth import FastLanguageModel

# ONLY save merged model (skip GGUF)
print("💾 Saving merged model...")

model.save_pretrained_merged(
    "n8n_workflow_generator_final",
    tokenizer,
    save_method = "merged_16bit",  # Full precision
)

print("✅ Model saved successfully!")
print("📁 Location: /content/n8n_workflow_generator_final/")

# Check what was saved
import os
files = os.listdir("n8n_workflow_generator_final")
print(f"\n📋 Files saved: {len(files)} files")
print("Main files:", [f for f in files if f.endswith('.safetensors') or f.endswith('.json')][:5])

In [ ]:
from huggingface_hub import notebook_login, create_repo

# 1. Login (skip if already logged in)
# notebook_login()  # Comment out if already logged in

# 2. Create repository
username = "MustaphaL"  # Your HF username
repo_name = "n8n-workflow-generator"
full_repo_id = f"{username}/{repo_name}"  # ← This is what was missing!

print(f"📦 Creating repo: {full_repo_id}")
create_repo(full_repo_id, exist_ok=True, private=False)

# 3. Push model
print("☁️ Uploading model to Hugging Face...")

model.push_to_hub_merged(
    full_repo_id,  # ← Use full path here!
    tokenizer,
    save_method = "merged_16bit",
    token = True,
)

print(f"✅ Model uploaded!")
print(f"🔗 View at: https://huggingface.co/{full_repo_id}")